## Part 3 Loading Data & EDA

Import relevant packages for the following parts

In [1]:
import numpy as np
import pandas as pd
import math
import re
import sys

#from gensim.models.word2vec import Word2Vec

import matplotlib.pyplot as plt
%matplotlib inline
import preprocess

plt.style.use('ggplot')
#from preprocessing import get_processed_data, load_data
import csv
import stanza
from nltk.corpus import wordnet
import spacy

from tqdm import tqdm

import time

### Import data cleaned by using the selfwritten preprocessing module

In [1114]:
event_df = pd.read_csv('moria_no_duplicates.csv', index_col=0)

event_df.reset_index(drop=True, inplace=True)
# given event date, split the dataset to pre and post event dfs

event_date = '2020-09-09'

#moria_df[['Date','Time']] = moria_df['Date Short'].astype(str).str.split(' ', 1, expand=True)
# create pre and post event partition
pre_event = event_df[event_df['Date Short'] < event_date]
post_event = event_df[event_df['Date Short'] >= event_date]

print('total tweets: ', event_df.shape[0])
print('Pre event tweets: ',pre_event.shape[0])
print('Post event tweets: ',post_event.shape[0])





total tweets:  18203
Pre event tweets:  3311
Post event tweets:  14892


small preprocessing
* spelling correction
* word separation
* merging mentions occuring next to each other
* remove signatures



In [1135]:
import re

def remove_tweet_signatures(tweet):
    texts_to_remove = ["Greece has a deadly new migration policy and all of Europe is to blame",
                       "| The Guardian",
                       "| Photo via Evening Standard",
                       "| Greece",
                       "| DW News ",
                       "- @WashTimes",
                       "(Guardian) Story:"
                  ]
    for text in texts_to_remove:
        tweet = tweet.replace(text,"")
    return tweet

In [1136]:
import preprocessor as p

sample_df = event_df["Tweet Raw"][:100].apply(p.clean)
sample_df = sample_df.apply(lambda x: remove_tweet_signatures(x))
list(sample_df)

['@sztiv5 @Juliivan_ Yes, why? Why it wasnt good to apply for asylum in Greece, MAC, SER or CRO or BUL, together 3 EU members before HU? They must get help in the first safe country as asylum seeker,not in the 5th. No law says you can pick and choose and get it.',
 "@GoTurkey ISIS refuge. Wouldn't go to Turkey if I was paid. You're likely to get your head lopped off if you stray off the beaten track. Go to Christian Greece and be safe while enjoying a similar climate to islamic turkey",
 'Greece must improve refugee overcrowding, UN warns',
 '@ThisIsOzcan @Nervana_1 @EGozuguzelli 1/3 Law? Let the idle stuff. All rights of the Turkish minority in Greece were taken away. Where is the law? Refugees are not accepted into the EU. Where is the law? Western states divided the states in the Middle East and Africa for underground resources. Where is the law?',
 '@Juliivan_ @sztiv5 Anyway, how did the asylum seekers ended up at HU borders? They must have had a long journey through Greece, where 

In [1115]:
list(event_df["Tweet Raw"])[:100]

['@sztiv5 @Juliivan_ Yes, why? Why it wasn’t good to apply for asylum in Greece, MAC, SER or CRO or BUL, together 3 EU members before HU? They must get help in the first safe country as asylum seeker,not in the 5th. No law says you can pick and choose and get it.',
 "@GoTurkey ISIS refuge. Wouldn't go to Turkey if I was paid. You're likely to get your head lopped off if you stray off the beaten track. Go to Christian Greece and be safe while enjoying a similar climate to islamic turkey",
 'Greece must improve refugee overcrowding, UN warns https://t.co/UDM4GDMcmo',
 '@ThisIsOzcan @Nervana_1 @EGozuguzelli 1/3 Law? Let the idle stuff. All rights of the Turkish minority in Greece were taken away. Where is the law? Refugees are not accepted into the EU. Where is the law? Western states divided the states in the Middle East and Africa for underground resources. Where is the law?',
 '@Juliivan_ @sztiv5 Anyway, how did the asylum seekers ended up at HU borders? They must have had a long journ

In [4]:
#stanza.download("en")

In [252]:
en_nlp = stanza.Pipeline("en", ner_batch_size=2048)

2021-03-06 11:36:15 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-03-06 11:36:15 INFO: Use device: cpu
2021-03-06 11:36:15 INFO: Loading: tokenize
2021-03-06 11:36:15 INFO: Loading: pos
2021-03-06 11:36:15 INFO: Loading: lemma
2021-03-06 11:36:15 INFO: Loading: depparse
2021-03-06 11:36:16 INFO: Loading: sentiment
2021-03-06 11:36:17 INFO: Loading: ner
2021-03-06 11:36:17 INFO: Done loading processors!


In [7]:
start = time.time()
en_doc = event_df["Tweet Raw"][:100].apply(en_nlp)
end = time.time()
print(f"Preprocessing the data took {end-start} seconds.")

Preprocessing the data took 168.26420331001282 seconds.


In [10]:
en_doc.iloc[0]

[
  [
    {
      "id": 1,
      "text": "@sztiv5",
      "lemma": "@sztiv5",
      "upos": "PROPN",
      "xpos": "NNP",
      "feats": "Number=Sing",
      "head": 0,
      "deprel": "root",
      "misc": "start_char=0|end_char=7",
      "ner": "O"
    },
    {
      "id": 2,
      "text": "@",
      "lemma": "@",
      "upos": "ADP",
      "xpos": "IN",
      "head": 3,
      "deprel": "case",
      "misc": "start_char=8|end_char=9",
      "ner": "O"
    },
    {
      "id": 3,
      "text": "Juliivan_",
      "lemma": "Juliivan_",
      "upos": "PROPN",
      "xpos": "NNP",
      "feats": "Number=Sing",
      "head": 1,
      "deprel": "nmod",
      "misc": "start_char=9|end_char=18",
      "ner": "S-PERSON"
    }
  ],
  [
    {
      "id": 1,
      "text": "Yes",
      "lemma": "yes",
      "upos": "INTJ",
      "xpos": "UH",
      "head": 3,
      "deprel": "discourse",
      "misc": "start_char=19|end_char=22",
      "ner": "O"
    },
    {
      "id": 2,
      "text": ",",
    

In [28]:
#directory = '../../export CORENLP_HOME=' ##ADD DIRECTORY HERE
stanza.install_corenlp()

#import os
#os.environ["CORENLP_HOME"] = directory

2021-03-04 16:34:39 WARNING: Directory C:\Users\nikodemicek\stanza_corenlp already exists. Please install CoreNLP to a new directory.


## As initial WCL candidates, we extract coreference chains and noun phrases (NPs).

### SOME PREPROCESSING NEEDED
* remove links
* remove/merge mentions?
* remove recurring texts (signatures of news media)
* remove posts of some accounts (refugee_list)
* remove # from hashtags?
* exclude NERs that tag numbers - should we mark phrase as NE if the head is not NE?
* play around with candidate types
* optimize code and make it neater


Signatures of media:
"DW News - https://t.co/dLNMddqtBw" - #Budapest #EU #Europe #European #EuropeanUnion #German #Greece #Turkey
"| Photo via Evening Standard"  https://t.co/hrcDUI72DF


In [6]:
from stanza.server import CoreNLPClient

# get noun phrases with tregex
def noun_phrases(_client, tweet, _annotators=None):
    """
    Input: _client = CoreNLPClient instance
           _text = tweet text
           _annotators = allowed CoreNLP operations
    Output: list of all noun phrases in the tweet
    """
    pattern = 'NP'
    matches = _client.tregex(tweet,pattern,annotators=_annotators)

    return [sentence[match_id]['spanString'] for sentence in matches['sentences'] for match_id in sentence]


In [8]:
noun_phrase_list = []
with CoreNLPClient(timeout=300000, memory='16G') as client:
    for tweet in tqdm(event_df["Tweet Raw"]):
        noun_phrase = noun_phrases(client,str(tweet),_annotators="tokenize,ssplit,pos,lemma,parse,ner,coref")
        noun_phrase_list.append(noun_phrase)
        #noun_phrases(client,event_df["Tweet Raw"][:100],_annotators="tokenize,ssplit,pos,lemma,parse")

2021-02-24 10:01:34 INFO: Writing properties to tmp file: corenlp_server-799910cec1b243da.props
2021-02-24 10:01:34 INFO: Starting server with command: java -Xmx16G -cp export CORENLP_HOME=/path/to/stanford-corenlp-4.1.0\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 300000 -threads 5 -maxCharLength 100000 -quiet False -serverProperties corenlp_server-799910cec1b243da.props -preload -outputFormat serialized
100%|██████████████████████████████████████████████████████████████████████████| 18203/18203 [3:00:23<00:00,  1.68it/s]


In [128]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ THIS IS A TEST CODE ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

with CoreNLPClient(timeout=300000, memory='16G') as client:
    #ann = client.annotate(event_df["Tweet Raw"][:10])  
    noun_phrase_list = []
    for tweet in tqdm(event_df["Tweet Raw"][:3]):
        doc = en_nlp(tweet)

        noun_phrase,np_head = noun_phrases(client,str(tweet),doc, _annotators="tokenize,ssplit,pos,lemma,parse,ner,coref")
        noun_phrase_list.append((noun_phrase,np_head))

noun_phrase_list

2021-03-04 21:23:48 INFO: Writing properties to tmp file: corenlp_server-f4d04816a03c4a14.props
2021-03-04 21:23:48 INFO: Starting server with command: java -Xmx16G -cp C:\Users\nikodemicek\stanza_corenlp\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 300000 -threads 5 -maxCharLength 100000 -quiet False -serverProperties corenlp_server-f4d04816a03c4a14.props -preload -outputFormat serialized
 10%|████████▎                                                                          | 1/10 [01:10<10:33, 70.39s/it]

@sztiv5 @Juliivan_ Yes, why? Why it wasn’t good to apply for asylum in Greece, MAC, SER or CRO or BUL, together 3 EU members before HU? They must get help in the first safe country as asylum seeker,not in the 5th. No law says you can pick and choose and get it.


 20%|████████████████▌                                                                  | 2/10 [01:15<06:47, 50.88s/it]

@GoTurkey ISIS refuge. Wouldn't go to Turkey if I was paid. You're likely to get your head lopped off if you stray off the beaten track. Go to Christian Greece and be safe while enjoying a similar climate to islamic turkey


 30%|████████████████████████▉                                                          | 3/10 [01:17<04:13, 36.17s/it]

...

 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [01:45<00:08,  8.13s/it]

9PM Coronavirus UK LIVE: Leeds on brink of lockdown, outbreak at asylum seeker hostel in Birmingham and Greece chaos Stories from 2 sources | Photo via Evening Standard https://t.co/hrcDUI72DF #birmingham #coronavirus #coronavirusoutbreak https://t.co/FOn9kkanHn


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:49<00:00, 10.99s/it]

@Iceman070259590 @OPCW @realDonaldTrump @mikepompeo Are you sure you' re talking about France and not Turkey? Turkey has invaded parts of Syria, bombs Iraq, conducts a proxy war in Libya creating refugees. On top of that, it violates Greece's and Cyprus' sovereignties. Not to mention that it provides safe heaven for Hamas.


[(['@sztiv5 @Juliivan_ Yes, why?',
   '@sztiv5',
   '@Juliivan_ Yes, why',
   '@Juliivan_',
   'it',
   'asylum in Greece, MAC, SER or CRO or BUL',
   'asylum in Greece, MAC, SER or CRO',
   'asylum',
   'Greece, MAC, SER or CRO',
   'BUL',
   '3 EU members',
   'HU',
   'They',
   'help',
   'the first safe country as asylum seeker',
   'the first safe country',
   'asylum seeker',
   'the 5th',
   'No law',
   'you',
   'it'],
  'says'),
 (['@GoTurkey ISIS',
   'refuge',
   'Turkey',
   'I',
   'You',
   'your head',
   'you',
   'the beaten track',
   'Christian Greece',
   'a similar climate',
   'islamic turkey'],
  'Go'),
 (['Greece', 'refugee overcrowding', 'UN', 'https://t.co/UDM4GDMcmo'],
  'improve'),
 (['@ThisIsOzcan @Nervana_1 @EGozuguzelli',
   '1/3',
   'Law',
   'the idle stuff',
   'All rights of the Turkish minority in Greece',
   'All rights',
   'the Turkish minority in Greece',
   'the Turkish minority',
   'Greece',
   'the law',
   'Refugees',
   'the EU',
   'the

In [1137]:

def tag_tweets(corpus):
    """
    Input: corpus of tweets to tag
    Output: List of tuples containing (POS-tags of each word, NER-tags of each named entity)
    """
    np_pos_tags=[]
    for tweet in tqdm(corpus):
        #for np in np_tweets:
            #annotate the tweet
            doc = en_nlp(tweet)
            #extract POS and NE tags
            tweet_pos_tags={word.text: word.xpos for sent in doc.sentences for word in sent.words}
            tweet_ner= {ent.text: ent.type for sent in doc.sentences for ent in sent.ents}
            np_pos_tags.append((tweet_pos_tags,tweet_ner))
    return np_pos_tags  

np_pos_tags = tag_tweets(sample_df) 



pos_tags_set = set()

#get a set of all NER tags existing in corpus
for tweet in np_pos_tags:
    tweet_pos_tags = set(tweet[1].values())
    pos_tags_set.update(tweet_pos_tags)

print(pos_tags_set)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [03:18<00:00,  1.99s/it]

{'QUANTITY', 'MONEY', 'WORK_OF_ART', 'EVENT', 'TIME', 'LOC', 'PERCENT', 'GPE', 'FAC', 'CARDINAL', 'PRODUCT', 'PERSON', 'ORDINAL', 'NORP', 'ORG', 'DATE'}


In [7]:
#Store the noun phrases in the pickle file
import pickle

with open('file_name_to_save', 'wb') as fp:
    pickle.dump(noun_phrase_list, fp)

NameError: name 'noun_phrase_list' is not defined

In [1138]:
# Load NPs from pickle file
import pickle

with open(r"moria_noun_phrases", "rb") as input_file:
    noun_phrase_list = pickle.load(input_file)

def get_np_len(np_list):
    # calculates number of noun phrases in the corpus
    sum_len = 0
    for tweet_nps in noun_phrase_list:
        sum_len += len(tweet_nps)
    return sum_len
#noun_phrase_list

## Keep only NPs shorter than 20 words and parent NPs 

In [1139]:
print(get_np_len(noun_phrase_list))
for tweet_nps in noun_phrase_list:
    #reverse the list of tweets nps so we avoid moving indexes and leaving out some phrases 
    for np in reversed(tweet_nps):
        i = tweet_nps.index(np)
        np_split = np.split()
        if len(np_split) > 19:
            tweet_nps.remove(tweet_nps[i])
            
print(get_np_len(noun_phrase_list))

250785
248025


In [1140]:
#silly but easy way to remove the child NP and keep only parents, run until the sum_len stops decreasing
after_removal_len = 0
while after_removal_len != get_np_len(noun_phrase_list):
    after_removal_len = get_np_len(noun_phrase_list)
    for tweet_nps in noun_phrase_list:
        for np in range(len(tweet_nps)):
            try:
                #if the subsequent np (child np) is contained in the current one, remove the child np
                if tweet_nps[np].find(tweet_nps[np+1]) != -1:
                    tweet_nps.remove(tweet_nps[np+1])

            #ignore the error caused with end of the list
            except IndexError:
                pass

print(get_np_len(noun_phrase_list))

129322


## Assign candidate types to candidates

In [1141]:
#dictionary to assign candidate types based on named entities and part of speech tags
#the key tuple consists of (isNE, lexicographer type, plural)
cand_types = {(True,'PERSON',None):'person-ne',
              (True,'NORP',None):'person-ne',
              (True,'PERSON','plural'):'person-nes',
              (True,'NORP','plural'):'person-nes',
              (False,'PERSON',None):'person-nn',
              (False,'PERSON','plural'):'person-nns',
              (True,'ORG',None):'group-ne',
              (True,'FAC',None):'group-ne',
              (False,'ORG',None):'group',
              (True,'LOC',None):'loc-ne',
              (True,'GPE',None):'loc-ne',
              (False, 'LOC',None):'loc'
            }


In [1142]:
import nltk
from nltk.corpus import wordnet as wn


def get_synt_category(head):
    """
    Input: head word of the noun phrase e.g. 'aliens' from NP 'Illegal aliens' 
    Output: syntactic category of the head word as categorized using worndet
    """
    
    person_ss = wn.synsets("person")[0]
    #group_ss = wn.synsets("facility")[0]    
    place_ss = wn.synsets("location")[0]
    org_ss = wn.synsets("organization")[0]
    counter = 0
    synt_category = head
    try:
        while synt_category not in [None,'PERSON','LOC','ORG']:
            # words without meaning return empty lists and cause infinite loop, we need to throw error
            assert len(wn.synsets(synt_category))>0, f"{synt_category} has no synonyms"
            
            for ss in wn.synsets(synt_category):
                counter += 1                
                #print(ss.lemmas())
                #for hyper in ss.hypernyms():
                assert len(ss.hypernyms())>0, f"{ss} has no hypernyms"
                hyper = ss.hypernyms()[0]
                
                #print(f'for {synt_category} synonyms are: {ss}, hypernyms are: {hyper}')
                #print(f'synonym with person: {ss.wup_similarity(person_ss)}')
                #print(f'hypernym with person: {hyper.wup_similarity(person_ss)}')
                #print(f'with group: {ss.wup_similarity(group_ss)}')
                #print(f'synonym with place: {ss.wup_similarity(place_ss)}')
                #print(f'hypernym with place: {hyper.wup_similarity(place_ss)}')

                #if the syntactic similarity to one of the categories is more than 0.7, select the category
                if ss.wup_similarity(person_ss) >= 0.7:
                    synt_category = 'PERSON'
                    break
                #elif ss.wup_similarity(group_ss) >= 0.7:
                    #synt_category = 'facility'
                    #break
                elif ss.wup_similarity(place_ss) >= 0.7:
                    synt_category = 'LOC'
                    break
                elif ss.wup_similarity(org_ss) >= 0.7:
                    synt_category = 'ORG'
                    break
                else:
                    # if the synset is not similar assign the hypernym synset
                    synt_category = hyper.lemma_names()[0]

                #force stop at level 5 of hypernym search
                if counter == 5:
                    synt_category = None
                    break
            
    except AssertionError:
        synt_category = None
        return synt_category

    #print(f'{head} turned into a candidate {synt_category}')  
    
    return synt_category


#test syntactic categories on list of noun phrases
results = Counter(word for tweet_nps in noun_phrase_list[:100] for np in tweet_nps for word in np.split())
#print(results)
list_of_cand_types = []
for head in results.keys():
    list_of_cand_types.append(get_synt_category(head))
    
count_types = Counter(list_of_cand_types)
print(count_types)

Counter({None: 1015, 'PERSON': 83, 'ORG': 51, 'LOC': 45})


## assign candidate type to noun phrases

In [1143]:
def get_cand_type(candidate, WCLcands = noun_phrase_list):
    """
    Input: list of all noun phrases occurring in one tweet
    Output: list of pairs of np (string) and its candidate type (string) in a tuple for each np of the tweet
    """
    i = WCLcands.index(candidate)
    np_cand_type = []
    for np in candidate:
        
        #annotate the noun phrase to find noun head (((((could be done before?)))))
        doc = en_nlp(np)  
        
        #the head of noun phrase is marked with value 0 for the word.head
        np_heads = {word.text: word.head for sent in doc.sentences for word in sent.words}
        for word, head in np_heads.items():  
            if head == int(0):
                np_head = word
        
        #print(f'the head of "{np}" is {np_head}')
        
        #check if the noun phrase contains an NE tag
        # possible problem - if three is marked as NE then three children will be a NE, should they be???????????
        isNE = False
        #print(np_pos_tags[i])
        for key in np_pos_tags[i][1].keys():
            if key in np:
                isNE = True                
        
        # identified entity will be none if the head is not a named entity, if it is, the NER tag will be assigned
        ner_tag = None
        
        for key in np_pos_tags[i][1].keys():
            if np_head in key:
                ner_tag = np_pos_tags[i][1][key]
        #if np_head in np_pos_tags[i][1].keys():
        #   identified_ner = np_pos_tags[i][1][np_head]       
        
        identified_ner = ner_tag if ner_tag != None else get_synt_category(np_head)
            
        #print(np_pos_tags[i])
        pos_number = None
        if np_head in np_pos_tags[i][0].keys():
            pos_tag = np_pos_tags[i][0][np_head]
            pos_number = 'plural' if pos_tag in ['NNS','NNPS'] and identified_ner in ['person','PERSON'] else None
        
        #we want to create a tuple of (is_named_entity, NE_tag/synt_category, POS-tag)
        pre_cand_type = (isNE, identified_ner, pos_number)
        
        #print(f'\n isNE: {isNE}, ner: {identified_ner}, pos: {pos_number}')
        cand_type = cand_types[pre_cand_type] if pre_cand_type in cand_types.keys() else 'misc'
        np_cand_type_pair = (np,np_head,cand_type)
        #print(np_cand_type_pair)
        np_cand_type.append(np_cand_type_pair)
    return np_cand_type



In [1144]:
np_and_cand_list = []
#print(noun_phrase_list[10])
for tweet_nps in tqdm(noun_phrase_list[:100]):
    np_and_cand_list.append(get_cand_type(tweet_nps))
print(np_and_cand_list)   

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:16<00:00,  2.57s/it]

[[('@sztiv5 @Juliivan_ Yes, why?', 'why', 'misc'), ('it', 'it', 'misc'), ('asylum in Greece, MAC, SER or CRO or BUL', 'asylum', 'group-ne'), ('3 EU members', 'members', 'person-nes'), ('HU', 'HU', 'group-ne'), ('They', 'They', 'misc'), ('help', 'help', 'person-nn'), ('the first safe country as asylum seeker', 'country', 'group-ne'), ('the 5th', '5th', 'misc'), ('No law', 'law', 'misc'), ('you', 'you', 'misc'), ('it', 'it', 'misc')], [('@GoTurkey ISIS', '@GoTurkey', 'misc'), ('refuge', 'refuge', 'loc'), ('Turkey', 'Turkey', 'loc-ne'), ('I', 'I', 'group'), ('You', 'You', 'misc'), ('your head', 'head', 'person-nn'), ('the beaten track', 'track', 'loc'), ('Christian Greece', 'Christian', 'person-ne'), ('a similar climate', 'climate', 'misc'), ('islamic turkey', 'turkey', 'misc')], [('Greece', 'Greece', 'loc-ne'), ('refugee overcrowding', 'overcrowding', 'group'), ('UN', 'UN', 'group-ne'), ('https://t.co/UDM4GDMcmo', 'https://t.co/UDM4GDMcmo', 'misc')], [('@ThisIsOzcan @Nervana_1 @EGozuguze

In [1145]:
from collections import Counter

cand_type_count = [np[1] for nps in np_and_cand_list for np in nps]
    
counts = Counter(cand_type_count)
print(counts)

Counter({'Greece': 43, 'it': 16, 'they': 15, 'you': 14, 'refugees': 14, 'I': 12, 'EU': 8, '#': 8, 'people': 8, 'asylum': 6, 'Turkey': 6, 'It': 6, 'them': 6, 'camp': 6, 'this': 5, 'visit': 5, 'we': 5, 'This': 5, 'He': 5, 'thousands': 5, 'migrants': 5, 'They': 4, 'country': 4, 'law': 4, 'countries': 4, 'Lesbos': 4, 'Covid': 4, 'fears': 4, 'immigrants': 4, 'There': 4, 'solidarity': 4, 'sea': 4, 'help': 3, 'refuge': 3, 'turkey': 3, 'seekers': 3, 'borders': 3, 'Europe': 3, 'Commissioners': 3, 'work': 3, 'applications': 3, 'challenges': 3, 'he': 3, 'Moria': 3, 'territory': 3, '@tomyork5': 3, '#refugees': 3, 'conditions': 3, 'We': 3, 'one': 3, '%': 3, 'case': 3, 'members': 2, 'You': 2, 'climate': 2, 'UN': 2, 'Refugees': 2, 'states': 2, 'The': 2, 'Guardian': 2, 'outbreak': 2, 'Raquel': 2, 'parents': 2, 'crisis': 2, 'numbers': 2, 'States': 2, 'border': 2, 'me': 2, 'problem': 2, 'life': 2, 'Agency': 2, 'tension': 2, 'immigration': 2, 'migration': 2, 'Germany': 2, 'end': 2, 'support': 2, 'there':

## If wordnet is not working well, try sense2vec and spacy

In [40]:
import spacy
from sense2vec import Sense2VecComponent

nlp = spacy.load("en_core_web_sm")
s2v = nlp.add_pipe("sense2vec")
s2v.from_disk(r"C:\Users\nikodemicek\Dropbox (CBS)\Master thesis data\s2v_old")


#vector_map.load(r"C:\Users\nikodemicek\Dropbox (CBS)\Master thesis data\s2v_old")


In [63]:
from sense2vec import Sense2Vec
s2v = Sense2Vec().from_disk(r"C:\Users\nikodemicek\Dropbox (CBS)\Master thesis data\s2v_old")
vector = s2v["natural_language_processing|NOUN"]
most_similar = s2v.most_similar("animal|NOUN", n=10)


In [64]:
most_similar


[('wild_animal|NOUN', 0.8854),
 ('living_animal|NOUN', 0.8663),
 ('animals|NOUN', 0.863),
 ('domesticated_animal|NOUN', 0.8566),
 ('living_creature|NOUN', 0.8442),
 ('other_animals|NOUN', 0.8427),
 ('other_animal|NOUN', 0.8415),
 ('farm_animal|NOUN', 0.8369),
 ('live_animal|NOUN', 0.8266),
 ('single_animal|NOUN', 0.8223)]

## Comparison of NER performance using SpaCy language module

In [22]:
import spacy


en_nlp = spacy.load("en_core_web_sm")


spacy_ner=[]
for np_tweets in event_df["Tweet Raw"][:50]:
    #for np in np_tweets:
        doc = en_nlp(np_tweets)
        #ner = en_nlp.add_pipe("ner")
        tweet_pos_tags={token.text: token.tag_ for token in doc}
        tweet_ner= {ent.text: ent.label_ for ent in doc.ents}
        #tweet_pos_tags={word.text: word.xpos for sent in doc.sentences for word in sent.words}
        #tweet_ner= {ent.text: ent.type for sent in doc.sentences for ent in sent.ents}
        spacy_ner.append((tweet_pos_tags,tweet_ner))
        
spacy_ner

[({'@sztiv5': 'NN',
   '@Juliivan': 'NNP',
   '_': 'NN',
   'Yes': 'UH',
   ',': ',',
   'why': 'WRB',
   '?': '.',
   'Why': 'WRB',
   'it': 'PRP',
   'was': 'VBD',
   'n’t': 'RB',
   'good': 'JJ',
   'to': 'TO',
   'apply': 'VB',
   'for': 'IN',
   'asylum': 'NN',
   'in': 'IN',
   'Greece': 'NNP',
   'MAC': 'NNP',
   'SER': 'NNP',
   'or': 'CC',
   'CRO': 'NNP',
   'BUL': 'NNP',
   'together': 'RB',
   '3': 'CD',
   'EU': 'NNP',
   'members': 'NNS',
   'before': 'IN',
   'HU': 'NNP',
   'They': 'PRP',
   'must': 'MD',
   'get': 'VB',
   'help': 'NN',
   'the': 'DT',
   'first': 'JJ',
   'safe': 'JJ',
   'country': 'NN',
   'as': 'IN',
   'seeker': 'NN',
   'not': 'RB',
   '5th': 'NN',
   '.': '.',
   'No': 'DT',
   'law': 'NN',
   'says': 'VBZ',
   'you': 'PRP',
   'can': 'MD',
   'pick': 'VB',
   'and': 'CC',
   'choose': 'VB'},
  {'@Juliivan': 'PERSON',
   'Greece': 'GPE',
   'MAC': 'ORG',
   'SER': 'ORG',
   'CRO': 'ORG',
   'BUL': 'ORG',
   '3': 'CARDINAL',
   'EU': 'ORG',
   'H

In [178]:
pos_tags_set = set()
for tweet in spacy_ner:
    tweet_pos_tags = set(tweet[1].values())
    pos_tags_set.update(tweet_pos_tags)

print(len(spacy_ner))
for nm in range(len(spacy_ner)):
    
    #print(event_df["Tweet Raw"][nm])
    

    #print(xposses)
    print(np_xpos[nm][1])
    print('\n\n')

50
{'Juliivan_': 'PERSON', 'Greece': 'GPE', 'MAC': 'ORG', 'SER': 'ORG', 'CRO': 'ORG', 'BUL': 'ORG', '3': 'CARDINAL', 'EU': 'ORG', 'HU': 'ORG', 'first': 'ORDINAL', 'the 5th': 'DATE'}



{'ISIS': 'ORG', 'Turkey': 'GPE', 'Christian': 'NORP'}



{'Greece': 'GPE', 'UN': 'ORG'}



{'1': 'CARDINAL', 'Turkish': 'NORP', 'Greece': 'GPE', 'EU': 'ORG', 'Western': 'NORP', 'the Middle East': 'LOC', 'Africa': 'LOC'}



{'HU': 'ORG', 'Greece': 'GPE'}



{'#Lesbos': 'FAC', 'Moria #migrants camp': 'FAC'}



{'GreeceMFA': 'ORG', 'Manfred': 'PERSON', 'Weber': 'PERSON', 'Europe': 'LOC', 'Germany': 'GPE', 'Greece': 'GPE', 'German': 'NORP', 'MFA': 'ORG'}



{'Refugee Covid case': 'EVENT', 'Guardian': 'ORG'}



{'9PM': 'TIME', 'UK': 'GPE', 'Leeds': 'GPE', 'Birmingham': 'GPE', 'Greece': 'GPE', '2': 'CARDINAL'}



{'Trump': 'PERSON', 'France': 'GPE', 'Turkey': 'GPE', 'Syria': 'GPE', 'Iraq': 'GPE', 'Libya': 'GPE', 'Greece': 'GPE', 'Cyprus': 'GPE', 'Hamas': 'ORG'}



{'Raquel Bessudo': 'PERSON', 'Isaac Bessudo': 

## Get coreference chains from the tweet corpus

In [238]:
def get_coref_chain(tweet,client):

    ann = client.annotate(tweet)        
    tweet_chains = ann.corefChain
    all_chains = list()
    
    
    for chain in tweet_chains:
        mychain = list()
        # Loop through every mention of this chain
        for mention in chain.mention:
            # Get the sentence in which this mention is located, and get the words which are part of this mention
            words_list = ann.sentence[mention.sentenceIndex].token[mention.beginIndex:mention.endIndex]
            #build a string out of the words of this mention
            ment_word = ' '.join([x.word for x in words_list])
            
            mychain.append(ment_word)
            
        #the corefering words will be stored alongside the index of their representative in a tuple
        coref_group = (mychain,chain.representative)
        all_chains.append(coref_group)
    return all_chains


dict_of_tweet_corefs = {}
with CoreNLPClient(properties={'annotators': 'coref', 'coref.algorithm' : 'statistical'}, memory='16G') as client:
    for tweet in tqdm(event_df["Tweet Raw"]):
        tweet_corefs=[]
        #print(f'Coreferences for the tweet {list(event_df["Tweet Raw"]).index(tweet)} are:')
        for chain in get_coref_chain(tweet,client):
            tweet_corefs.append(chain)
            #print(' <-> '.join(chain),'\n')
        dict_of_tweet_corefs[list(event_df["Tweet Raw"]).index(tweet)] = tweet_corefs

2021-02-25 22:28:20 INFO: Writing properties to tmp file: corenlp_server-09eec83ac96340ea.props
2021-02-25 22:28:20 INFO: Starting server with command: java -Xmx16G -cp export CORENLP_HOME=/path/to/stanford-corenlp-4.1.0\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet False -serverProperties corenlp_server-09eec83ac96340ea.props -preload -outputFormat serialized
100%|████████████████████████████████████████████████████████████████████████████| 18203/18203 [57:38<00:00,  5.26it/s]


In [26]:
# FOR TESTING PURPOSES
all_chains = []
for chain in tweet_chains:
        mychain = list()
        # Loop through every mention of this chain
        for mention in chain.mention:
            # Get the sentence in which this mention is located, and get the words which are part of this mention
            words_list = ann.sentence[mention.sentenceIndex].token[mention.beginIndex:mention.endIndex]
            #build a string out of the words of this mention
            ment_word = ' '.join([x.word for x in words_list])
            #chain_rep = chain.representative
            #coref_group = (ment_word,chain_rep)
            
            mychain.append(ment_word)
            
        coref_group = (mychain,chain.representative)
        all_chains.append(coref_group)
        
all_chains

NameError: name 'tweet_chains' is not defined

In [239]:
with open('moria_tweet_corefs', 'wb') as fp:
    pickle.dump(dict_of_tweet_corefs, fp)

dict_of_tweet_corefs

{0: [(['No law', 'it'], 0)],
 1: [(['You', 'your', 'you'], 0)],
 2: [],
 3: [(['the law', 'the law', 'the law'], 0)],
 4: [(['They', 'they', 'the asylum seekers', 'they'], 2)],
 5: [(['#photograpy pushing aesthetic', 'it'], 0), (['I', 'I'], 0)],
 6: [(['@GreeceMFA ManfredWeber', 'It'], 0)],
 7: [],
 8: [],
 9: [(['Turkey', 'Turkey', 'it', 'it'], 0), (['you', 'you'], 0)],
 10: [(['Raquel',
    'Raquel',
    '@realPR_Phoenix @DrEstella @BravoTV Raquel Bessudo',
    'Isaac Bessudo'],
   2)],
 11: [(['@Mproyklis @fragoua @LearnerLerner', 'them', 'their'], 0)],
 12: [(['you', 'you', 'your', 'your', 'you'], 1),
  (['it', 'it'], 1),
  (['they', 'they'], 0)],
 13: [],
 14: [(['It', 'Greece'], 1)],
 15: [],
 16: [],
 17: [(['their', 'Both men', 'they', 'them', 'they'], 1)],
 18: [(['Those refugees',
    '10 refugees , including three children with disabilities'],
   1),
  (['Greece', 'Greece'], 0)],
 19: [(['Anadolu Agency', 'UN Refugee Agency'], 1)],
 20: [],
 21: [],
 22: [],
 23: [(['we', 'w

## Determining candidate's type for corefs

In [362]:
# Load NPs from pickle file
import pickle

with open(r"moria_tweet_corefs", "rb") as input_file:
    corefs = pickle.load(input_file)



In [363]:
corefs_list = []
for tweet_corefs in corefs:
    try:
        tw_corefs = [coref[0][coref[1]] for coref in corefs[tweet_corefs]]        
        corefs_list.append(tw_corefs)
    except IndexError:
        corefs_list.append([])

        
corefs_list       
#corefs[9][0][0][corefs[0][0][1]]

[['No law'],
 ['You'],
 [],
 ['the law'],
 ['the asylum seekers'],
 ['#photograpy pushing aesthetic', 'I'],
 ['@GreeceMFA ManfredWeber'],
 [],
 [],
 ['Turkey', 'you'],
 ['@realPR_Phoenix @DrEstella @BravoTV Raquel Bessudo'],
 ['@Mproyklis @fragoua @LearnerLerner'],
 ['you', 'it', 'they'],
 [],
 ['Greece'],
 [],
 [],
 ['Both men'],
 ['10 refugees , including three children with disabilities', 'Greece'],
 ['UN Refugee Agency'],
 [],
 [],
 [],
 ['we'],
 [],
 [],
 [],
 ['Enterprise Greece'],
 [],
 ['they'],
 ['He'],
 [],
 ['UK', 'UK size'],
 [],
 ['the camp', '@guardian The infected person , a 40 year old man', 'Moria'],
 [],
 ['Greece', 'Protection & Operations , @GillianTriggs & @RaoufMazou'],
 ['I', 'you'],
 [],
 [],
 [],
 [],
 [],
 ['you'],
 ['Turkey - Greece'],
 ['I'],
 [],
 [],
 ['Greece', 'freedom of expression', 'your'],
 [],
 ['2040', 'Europe'],
 ['People', 'me', 'this poor nation'],
 ['@AmysGotBirds @DonaldJTrumpJr', 'Syrian refugees'],
 [],
 [],
 [],
 ['@Nicolas98869551 @Marflo6

In [1146]:
coref_and_cand_list = []
for tweet_corefs in tqdm(corefs_list[:100]):
    coref_and_cand_list.append(get_cand_type(tweet_corefs,corefs_list))
print(coref_and_cand_list)  

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:16<00:00,  6.09it/s]

[[('No law', 'law', 'misc')], [('You', 'You', 'misc')], [], [('the law', 'law', 'misc')], [('the asylum seekers', 'seekers', 'person-nns')], [('#photograpy pushing aesthetic', 'photograpy', 'misc'), ('I', 'I', 'misc')], [('@GreeceMFA ManfredWeber', '@GreeceMFA', 'misc')], [], [], [('Turkey', 'Turkey', 'loc-ne'), ('you', 'you', 'misc')], [('@realPR_Phoenix @DrEstella @BravoTV Raquel Bessudo', 'Raquel', 'person-ne')], [('@Mproyklis @fragoua @LearnerLerner', '@Mproyklis', 'misc')], [('you', 'you', 'misc'), ('it', 'it', 'misc'), ('they', 'they', 'misc')], [], [('Greece', 'Greece', 'loc-ne')], [], [], [('Both men', 'men', 'group')], [('10 refugees , including three children with disabilities', 'refugees', 'person-nes'), ('Greece', 'Greece', 'loc-ne')], [('UN Refugee Agency', 'Agency', 'group-ne')], [], [], [], [('we', 'we', 'misc')], [], [], [], [('Enterprise Greece', 'Greece', 'group-ne')], [], [('they', 'they', 'misc')], [('He', 'He', 'misc')], [], [('UK', 'UK', 'loc-ne'), ('UK size', 'si

## Candidate merging

We organize candidates in a list sorted by their number of phrases

In [1147]:
#concatenate corefs and noun phrase lists
candidate_list = coref_and_cand_list + np_and_cand_list
#unpack list of lists into one list
candidate_list = [cand for cands in candidate_list for cand in cands]

In [1148]:
def get_cand_len(cand):
    #sort candidate list by count of words in the first element of the tuple
    return len(cand[0].split())

candidate_list.sort(reverse=True, key=get_cand_len) 

In [1149]:


def get_head(phrase):
    #annotate the noun phrase to find noun head (((((could be done before?)))))
    doc = en_nlp(phrase)  

    #the head of noun phrase is marked with value 0 for the word.head
    relations = {word.text: word.head for sent in doc.sentences for word in sent.words}
    for word, phrase in relations.items():  
        if phrase == int(0):
            np_head = word
            return np_head


### First merging step

In [1185]:
#
# THIS IS THE FIRST MERGING STEP
#
        
def merging_step1(candidate_list):
    """
    In the first merging step, we merge two candidates if the head of each of their representative phrase 
     is identical by string comparison.
    """
    indices_to_remove = set()
    for longer_cand in range(len(candidate_list)):     
        for cand in range(longer_cand+1,len(candidate_list)): 
            #print(f'for index {candidate_list[longer_cand][1]} checking the index {candidate_list[cand][1]}')

            #performing merging only for NE candidates of the same type
            if 'ne' in candidate_list[longer_cand][2]:
                #mark for merging if the head and its head's cand type is the same for 2 candidates
                if candidate_list[longer_cand][1] == candidate_list[cand][1] and candidate_list[longer_cand][2] == candidate_list[cand][2]:
                    print(f'matching "{longer_cand}" with "{cand}"')
                    indices_to_remove.add(cand)
    return indices_to_remove

def merge_indices(candidate_list,indices_to_remove):                
    print(len(candidate_list))                
    print(len(sorted(indices_to_remove)))

    for index in reversed(sorted(indices_to_remove)):
        candidate_list.remove(candidate_list[index])

    print(len(candidate_list)) 
    return candidate_list


candidate_list = merge_indices(candidate_list, merging_step1(candidate_list))

180
0
180


In [1050]:
candidate_list[136]

('Law', 'Law', 'misc')

### Second merging step

Simplification - we are taking the vector of the main phrase head instead of the mean of all heads

In [1051]:
import gensim

#load the GoogleNews 300dim model (fix path)
model = gensim.models.KeyedVectors.load_word2vec_format(r'C:\Users\nikodemicek\Desktop\GoogleNews-vectors-negative300.bin.gz', binary=True)

In [1179]:
#adjust for sets of phrases in the candidate
def merging_step2(candidate_list):
    
    indices_to_remove = set()
    for longer_cand in range(len(candidate_list)):     
        i = candidate_list[longer_cand]

        for cand in range(longer_cand+1,len(candidate_list)): 
            #print(f'for index {candidate_list.index(longer_cand)} checking the index {candidate_list.index(cand)}')
            if candidate_list[longer_cand][1] == candidate_list[cand][1]:
                print(f'matching "{longer_cand}" with "{cand}"')
            try:
                if candidate_list[longer_cand][2] == candidate_list[cand][2]:
                    if model.similarity(candidate_list[longer_cand][1], candidate_list[cand][1]) >= 0.5:
                        print(f'matching "{longer_cand}" with "{cand}"') 
                        #print(f'{longer_cand[1]} and  {cand[1]} matched with sim {model.similarity(longer_cand[1], cand[1])}')
                        indices_to_remove.add(cand)
                elif model.similarity(candidate_list[longer_cand][1], candidate_list[cand][1]) >= 0.7:
                    print(f'matching "{longer_cand}" with "{cand}"') 
                    #print(f'{longer_cand[1]} and  {cand[1]} matched with sim {model.similarity(longer_cand[1], cand[1])}')
                    indices_to_remove.add(cand)
                else:
                    pass
            except KeyError:
                pass
    return indices_to_remove

candidate_list = merge_indices(candidate_list, merging_step2(candidate_list))


247
0
247


In [1170]:
candidate_list[103]

('The #refugees in Greek islands', '#refugees', 'person-ne')

## Third merging step representative labeling

currently working on average cosine similarity of each phrase in the candidate - maybe not optimal, maybe it will be better with a different threshold

In [1181]:
from sklearn.cluster import AffinityPropagation
import numpy 
from sklearn.metrics.pairwise import cosine_similarity

def merging_step3(candidate_list):
    phrases = []
    indices_to_remove = set()
    # 1. first we find adj-nn phrases within the candidate
    for np in candidate_list:
        #annotate the noun phrase to find noun head (((((could be done before?)))))
        doc = en_nlp(np[0])  

        #the head of noun phrase is marked with value 0 for the word.head
        np_heads_pos = [(word.text, word.head, word.xpos) for sent in doc.sentences for word in sent.words]
        #np_pos_tags = {word.text: word.xpos for sent in doc.sentences for word in sent.words}
        #print(np_heads_pos)
        cand_np_phrases = []
        for word, head, pos in np_heads_pos:
            #head-1 because the pointer to head does not use 0 index
            if (pos == 'JJ' or pos=='VBN') and 'NN' in np_heads_pos[head-1][2]:
                cand_np_phrases.append(f'{word}_{np_heads_pos[head-1][0]}')
        phrases.append(cand_np_phrases)
        
    # 2. we compare the similarities of candidates' phrases
    for longer_cand in range(len(candidate_list)):     
        i = candidate_list[longer_cand]
        long_cand_vectors = phrases_vectors(phrases[longer_cand])
        if len(long_cand_vectors)==0:
            pass
        else:
            for cand in range(longer_cand+1,len(candidate_list)): 
                short_cand_vectors = phrases_vectors(phrases[cand])
                if len(short_cand_vectors)==0:
                    pass
                else:
                    sim_matrix = numpy.zeros((len(long_cand_vectors),len(short_cand_vectors)))
                    #print(sim_matrix)
                    for i in range(len(long_cand_vectors)):
                        for j in range(len(short_cand_vectors)):

                            sim_matrix[i][j] = cosine_similarity(long_cand_vectors[i].reshape(1,-1),short_cand_vectors[j].reshape(1,-1))

                                
                    if numpy.mean(sim_matrix) > 0.3:
                        #print(f'{longer_cand} and {cand} are {numpy.mean(sim_matrix)} similar' )
                        indices_to_remove.add(cand)
                    #else:
                        #print(f'{numpy.mean(sim_matrix)} is not similar' )
                    
    return indices_to_remove
                


def phrases_vectors(cand_phrases):
    
#for cand_phrases in phrases:
    #print(cand_phrases)
    cand_phrase_vectors = []
    for phrase in cand_phrases:
        try:
            cand_phrase_vectors.append(model[phrase])
            #print(f'for existing phrase "{phrase}" the vector is {model[phrase][0]}')
        except KeyError:
            phrase_words = phrase.split('_')
            #print(model[phrase_words[1]])
            try:
                phrase_vectors = [model[phrase_word] for phrase_word in phrase_words]
                #print(f'for phrase "{phrase}" avg vector is "{sum(phrase_vectors)/len(phrase_vectors)}') 
                cand_phrase_vectors.append(sum(phrase_vectors)/len(phrase_vectors))
            except KeyError:
                pass
    #print(len(cand_phrase_vectors))
    return cand_phrase_vectors
    
    
candidate_list = merge_indices(candidate_list, merging_step3(candidate_list))
#print(indices_to_remove)

194
0
194


In [1055]:
candidate_list[66]
    

('HU borders', 'borders', 'misc')

### Merging step 4

In [1183]:
# missing the second method - we check for the lexical identity of specific stems in multiple candidates.

def merging_step4(candidate_list):
    phrases = []
    indices_to_remove = set()
    # 1. first we find adj-nn phrases within the candidate
    for np in candidate_list:
        #annotate the noun phrase to find noun head (((((could be done before?)))))
        doc = en_nlp(np[0])  

        #the head of noun phrase is marked with value 0 for the word.head
        np_heads_pos = [(word.text, word.head, word.xpos) for sent in doc.sentences for word in sent.words]
        #np_pos_tags = {word.text: word.xpos for sent in doc.sentences for word in sent.words}
        #print(np_heads_pos)
        cand_np_phrases = []
        for word, head, pos in np_heads_pos:
            i = np_heads_pos.index((word, head, pos))
            #print(np_heads_pos)
            #print(np_heads_pos[i])
            #print(np_heads_pos[head-1])
            #'NN' in np_heads_pos[head-1][2] and
            try:
                if 'NN' in pos and 'NN' in np_heads_pos[i+1][2] : 
                    cand_np_phrases.append(f'{word}_{np_heads_pos[i+1][0]}')
                if 'NN' in pos and 'NN' in np_heads_pos[head-1][2]:
                    cand_np_phrases.append(f'{word}_{np_heads_pos[head-1][0]}')
            except IndexError:
                pass
        phrases.append(cand_np_phrases)
    
    # 2. we compare the similarities of candidates' phrases
    for longer_cand in range(len(candidate_list)):     
        i = candidate_list[longer_cand]
        long_cand_vectors = phrases_vectors(phrases[longer_cand])
        if len(long_cand_vectors)==0:
            pass
        else:
            for cand in range(longer_cand+1,len(candidate_list)): 
                short_cand_vectors = phrases_vectors(phrases[cand])
                if len(short_cand_vectors)==0:
                    pass
                else:
                    sim_matrix = numpy.zeros((len(long_cand_vectors),len(short_cand_vectors)))
                    #print(sim_matrix)
                    for i in range(len(long_cand_vectors)):
                        for j in range(len(short_cand_vectors)):
                            #print(cosine_similarity(long_cand_vectors[i].reshape(1,-1),short_cand_vectors[j].reshape(1,-1)))
                            sim_matrix[i][j] = cosine_similarity(long_cand_vectors[i].reshape(1,-1),short_cand_vectors[j].reshape(1,-1))
                            """if cosine_similarity(long_cand_vectors[i].reshape(1,-1),short_cand_vectors[j].reshape(1,-1)) > 0.4:                
                                sim_matrix[i][j] = 2
                            elif cosine_similarity(long_cand_vectors[i].reshape(1,-1),short_cand_vectors[j].reshape(1,-1)) > 0.2:
                                sim_matrix[i][j] = 1
                            else:
                                sim_matrix[i][j] = 0"""

                                
                    if numpy.mean(sim_matrix) > 0.6:
                        print(f'{longer_cand} and {cand} are {numpy.mean(sim_matrix)} similar' )
                        indices_to_remove.add(cand)
                    #else:
                        #print(f'{numpy.mean(sim_matrix)} is not similar' )
                    
    return indices_to_remove

candidate_list = merge_indices(candidate_list, merging_step4(candidate_list))
#print(merging_step4(candidate_list))

180
0
180


In [1184]:
candidate_list

[('conditions +reduce overcrowding at the reception facilities on the Aegean Islands - @RaoufMazou on #Greece #asylum #Europe #Solidarity https://t.co/H3yhHqly4g',
  'conditions',
  'misc'),
 ('population & children for 39%, of whom more than 7 out of 10 are younger than 12 years old',
  'population',
  'misc'),
 ('The largest terrorist organization in the world, the names of the Syrian refugees, to enter Bulgaria to Greece',
  'organization',
  'group-ne'),
 ('The first recorded coronavirus case in Moria refugee camp on Lesbos, where just under 13,000 people are l.....',
  'case',
  'misc'),
 ('the #Migration crisis | DW News - https://t.co/dLNMddqtBw - #Budapest #EU #Europe #European #EuropeanUnion #German #Greece #Turkey',
  'crisis',
  'misc'),
 ('Greece chaos Stories from 2 sources | Photo via Evening Standard https://t.co/hrcDUI72DF #birmingham #coronavirus #coronavirusoutbreak https://t.co/FOn9kkanHn',
  '#',
  'misc'),
 ('offered high-quality, #trauma-sensitive, identity-inform

### Merging step 5
